# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_csv = "../output_data/weather_data.csv"
city_df = pd.read_csv(weather_csv)
city_df

,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
0,53.8800,67.4158,256.40,88,34,2.69,KZ,1644185630
1,-0.7393,-90.3518,303.13,88,43,2.68,EC,1644185634
2,-23.1203,-134.9692,299.40,70,3,7.77,PF,1644185399
3,62.5500,155.5500,247.81,66,0,0.54,RU,1644185636
4,-34.6667,-54.9167,293.28,69,77,9.08,UY,1644185638
...,...,...,...,...,...,...,...,...
576,-3.4167,114.8500,297.20,94,100,1.03,ID,1644186454
577,-14.2426,31.3253,290.22,96,84,1.54,ZM,1644186456
578,-3.3667,135.4833,298.11,89,99,0.99,ID,1644186305
579,40.6167,120.7167,259.97,85,29,1.74,CN,1644186458


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# extract lat and long into own dataframe
location_df = city_df[["lat", "lng"]]

# extract humidity into series of floats
humidity = city_df["humidity"].astype(float)

In [16]:
# Plot Heatmap
fig = gmaps.figure()
# dataframe functions as our list of pairs (df with two columns)
heat_layer = gmaps.heatmap_layer(location_df,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal weather conditions: A max temperature lower than 80 (300 K) degrees but higher than 70 (294 K).
Wind speed less than 10 mph.
Zero cloudiness.

In [34]:
# narrow down dataframe based on above requirements
city_a = city_df.loc[(city_df["max temp"] > 294) & (city_df["max temp"] < 300)]
city_b = city_a.loc[(city_a["wind speed"] < 10)]
city_c = city_b.loc[(city_b["cloudiness"] == 0)]
final_city_df = city_c.dropna()
final_city_df.head()

,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
27,39.1502,-123.2078,297.66,28,0,0.00,US,1644185582
30,-25.2406,130.9889,295.15,37,0,3.47,AU,1644185672
37,-33.9180,25.5701,296.54,68,0,2.57,ZA,1644185682
50,34.6391,-120.4579,299.70,40,0,8.23,US,1644185600
89,-45.8667,-67.5000,297.07,35,0,4.12,AR,1644185700


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
hotel_df = final_city_df
hotel_df["hotel name"] = ""
hotel_df["city"] = ""
# reset index to make life easier when putting hotels in
hotel_df.reset_index(inplace=True)
hotel_df.head()

,index,lat,lng,max temp,humidity,cloudiness,wind speed,country,date,hotel name
0,27,39.1502,-123.2078,297.66,28,0,0.00,US,1644185582,
1,30,-25.2406,130.9889,295.15,37,0,3.47,AU,1644185672,
2,37,-33.9180,25.5701,296.54,68,0,2.57,ZA,1644185682,
3,50,34.6391,-120.4579,299.70,40,0,8.23,US,1644185600,
4,89,-45.8667,-67.5000,297.07,35,0,4.12,AR,1644185700,


In [88]:
# loop through all cities (just do 0-5 right now)
for index, hotel in enumerate(hotel_df.iterrows()):
    lat = hotel[1]['lat']
    lng = hotel[1]['lng']
    # set up params dictionary
    params = {
        "location": f"{lat}, {lng}",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response_json = requests.get(base_url, params=params).json()
    print(f"processing city number {index}")
    try:
        # first item after results seems to be city info so take second [1] item then grab hotel name
        hotel_df.loc[index, "hotel name"] = response_json['results'][1]['name']
        # get city name
        hotel_df.loc[index, "city"] = response_json['results'][0]['name']
    except:
        print(f"unable to find hotel for city number {index}")
hotel_df

processing city number 0
processing city number 1
processing city number 2
processing city number 3
processing city number 4
processing city number 5
processing city number 6
processing city number 7
processing city number 8
processing city number 9
processing city number 10
processing city number 11
processing city number 12
processing city number 13
processing city number 14
processing city number 15
processing city number 16
processing city number 17
processing city number 18
processing city number 19
processing city number 20
processing city number 21
processing city number 22
processing city number 23
processing city number 24
processing city number 25
processing city number 26
processing city number 27
processing city number 28
processing city number 29
processing city number 30


,index,lat,lng,max temp,humidity,cloudiness,wind speed,country,date,hotel name,city
0,27,39.1502,-123.2078,297.66,28,0,0.00,US,1644185582,Hampton Inn Ukiah,Ukiah
1,30,-25.2406,130.9889,295.15,37,0,3.47,AU,1644185672,Desert Gardens Hotel - Ayers Rock Resort,Petermann
2,37,-33.9180,25.5701,296.54,68,0,2.57,ZA,1644185682,Gelvandale Stadium,Gqeberha
3,50,34.6391,-120.4579,299.70,40,0,8.23,US,1644185600,"Holiday Inn Express Lompoc, an IHG Hotel",Lompoc
4,89,-45.8667,-67.5000,297.07,35,0,4.12,AR,1644185700,Austral Hotel,Comodoro Rivadavia
5,126,13.5728,-8.0339,294.81,10,0,4.34,ML,1644185806,Kolokani,Kolokani
6,157,-26.6167,-54.1333,298.03,36,0,1.84,AR,1644185856,Banco Macro Sucursal 13 San Pedro,San Pedro
7,167,17.0151,54.0924,298.20,31,0,4.63,OM,1644185665,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel",Salalah
8,192,-34.7333,135.8667,296.26,59,0,6.37,AU,1644185845,Port Lincoln Hotel,Port Lincoln
9,197,12.0708,1.7889,297.55,17,0,3.39,BF,1644185911,Haut commissariat de Diapaga,Diapaga


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [92]:
# Add marker layer ontop of heat map
# Assign the marker layer to locations and add info box
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))